# Circle Packing

## Introduction

We aim to pack some circles

In [1]:
HOME = "/home/coopar7/Documents/code/"
#HOME = "/home/jon/Documents/UBCMRI/"
cd(HOME * "BlochTorreyExperiments-master/")

In [254]:
using Traceur
using BenchmarkTools
using StaticArrays
using JuAFEM
using JuAFEM: vertices, faces, edges
using MATLAB
using LinearMaps
#using DifferentialEquations
using Optim
#using Cuba
using Distributions
#using ApproxFun
#using Plots
using ForwardDiff
using ReverseDiff

include("Experiments/MyelinWaterOrientation/Geometry/geometry_utils.jl")
include("Experiments/MyelinWaterOrientation/Geometry/circle_packing.jl")
include("Experiments/MyelinWaterOrientation/Utils/mesh_utils.jl")
include("Experiments/MyelinWaterOrientation/Utils/blochtorrey_utils.jl")
Revise.track("Experiments/MyelinWaterOrientation/Geometry/geometry_utils.jl")
Revise.track("Experiments/MyelinWaterOrientation/Geometry/circle_packing.jl")
Revise.track("Experiments/MyelinWaterOrientation/Utils/mesh_utils.jl")
Revise.track("Experiments/MyelinWaterOrientation/Utils/blochtorrey_utils.jl")

In [255]:
# α or k == R_shape, θ == R_scale
R_mu = 0.46 # Axon mean radius [um] ; this is taken to be outer radius
R_shape = 5.7 # Axon radius shape parameter for Gamma distribution (Xu)
R_scale = R_mu / R_shape # Axon radius scale parameter [um]
R_σ = sqrt(R_shape)*R_scale; # Axon radius variance

In [256]:
revise()

In [257]:
const Dim = 2
Ncircles = 50
rs = rand(Gamma(R_shape, R_scale), Ncircles);
os = initialize_origins(rs);

In [258]:
η = 0.80 # goal packing density
ϵ = 0.1*R_mu # overlap occurs when distance between circle edges is ≤ ϵ
α = 1.0 # density penalty weight
β = 1e-6 # mutual distance penalty weight
λ = 1.0 # overlap penalty weight (or lagrange multiplier for constrained version)
w = [α, β, λ] # vector of weights

@time circles_opt, opt_result = pack_circles(rs; initial_origins = os, goaldensity = η, distancescale = R_mu, weights = w, epsilon = ϵ);

ReverseDiff
 90.578670 seconds (103.84 M allocations: 2.802 GiB, 1.99% gc time)


In [232]:
opt_result;

In [ ]:
estimate_density(opt_result)

In [259]:
cs_plot = circles_opt;

In [260]:
estimate_density(cs_plot)

0.07589213495452453

In [235]:
revise()

In [236]:
Nmin = 50; # points for smallest circle
h0 = 2pi*mean(c->radius(c), cs_plot)/Nmin; # approximate scale
#h0 = ϵ
eta = 4.0; # approx ratio between largest/smallest edges
b_box = bounding_box(cs_plot)

Rectangle{2,Float64}([-4.65403, -1.74316], [12.851, 14.8568])

In [237]:
#FinElPath = HOME * "MatlabTools/FiniteElements/"
#mxcall(:addpath, 0, mxcall(:genpath, 1, FinElPath))

In [238]:
#fullgrid, subgrids = square_mesh_with_circles(b_box, cs_plot, h0, eta, isunion=true);

In [239]:
overlap_mat = zeros(Bool, Ncircles, Ncircles);
[overlap_mat[i,j] = is_overlapping(cs_plot[i], cs_plot[j]) for i in 1:Ncircles for j in 1:Ncircles]
overlap_mat;

In [240]:
dist_mat = zeros(Ncircles, Ncircles);
for i in 1:Ncircles-1, j in 1:i-1
    dist_mat[i,j] = signed_edge_distance(cs_plot[i], cs_plot[j])
end
min_dist = minimum(x->x==zero(x) ? Inf : x, dist_mat)
eps_dist = ϵ
@show min_dist
@show eps_dist
@show 100*(eps_dist - min_dist)/eps_dist;

159.330711 seconds (53.69 M allocations: 26.339 GiB, 2.45% gc time)


In [241]:
dist_mat;

min_dist = 0.03643476040850213
eps_dist = 0.046000000000000006
(100 * (eps_dist - min_dist)) / eps_dist = 20.793999111951898


In [ ]:
revise()